## Food2Fork API ##
www.food2fork.com/about/api

In [9]:
import re
import requests
import urllib2
from urllib import quote
import json
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
import nltk

In [10]:
%matplotlib inline

### API credentials ###

In [11]:
def loadCredentials():
    filename = 'secrets.txt'
    for line in open(filename).readlines():
        if "API" in line:    
            api_key = line.split(": ")[1].translate(None,'\n')
        
    return api_key

In [12]:
# Load the API Key
API_KEY = loadCredentials()

# Food2Fork links
_URL_API = "http://food2fork.com/api/"
_URL_KEY = "key={0}".format(API_KEY)
_URL_SEARCH = _URL_API + 'search?' + _URL_KEY
_URL_GET = _URL_API + 'get?' + _URL_KEY
_HEADER = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}

### Search the Food2Fork API ###

In [13]:
def search(query, page=1):   
    """Return a list of recipes from the Food2Fork.com database"""

    # Format the request URL    
    api_request = _URL_SEARCH+"&q={query}&page={page}".format(query=query,page=page)

    # Make the request
    request  = urllib2.Request(api_request, headers=_HEADER)
    response = urllib2.urlopen(request)
    raw = response.read()
    json_obj = json.loads(raw)  
    
    return json_obj['recipes']

In [14]:
# Search for chicken salad recipes
recipes = search('Chicken salad')
print(recipes[0]['image_url'])
recipes[0]

http://static.food2fork.com/Bacon2BWrapped2BJalapeno2BPopper2BStuffed2BChicken2B5002B5909939b0e65.jpg


{u'f2f_url': u'http://food2fork.com/view/35120',
 u'image_url': u'http://static.food2fork.com/Bacon2BWrapped2BJalapeno2BPopper2BStuffed2BChicken2B5002B5909939b0e65.jpg',
 u'publisher': u'Closet Cooking',
 u'publisher_url': u'http://closetcooking.com',
 u'recipe_id': u'35120',
 u'social_rank': 100.0,
 u'source_url': u'http://www.closetcooking.com/2012/11/bacon-wrapped-jalapeno-popper-stuffed.html',
 u'title': u'Bacon Wrapped Jalapeno Popper Stuffed Chicken'}

### Request a specific recipe from the API ###

In [15]:
def getRecipe(recipe_id):
    """Return the recipe specified by *recipe_id* from Food2Fork's database"""

    # Format the request URL
    api_request = _URL_GET+"&rId={ID}".format(ID=recipe_id)
    
    # Make the request
    request  = urllib2.Request(api_request, headers=_HEADER)
    response = urllib2.urlopen(request)
    raw = response.read()
    json_obj = json.loads(raw)  
    
    return json_obj['recipe']

In [16]:
r = getRecipe('35130')
r

{u'f2f_url': u'http://food2fork.com/view/35130',
 u'image_url': u'http://static.food2fork.com/BananasFosterPancakes15000eeed5d6.jpg',
 u'ingredients': [u'1 cup all purpose flour',
  u'1 teaspoon baking powder',
  u'1/4 teaspoon salt',
  u'1/4 teaspoon cinnamon',
  u'1 cup milk',
  u'1 egg',
  u'2 tablespoons brown sugar',
  u'2 tablespoons unsalted butter (melted)',
  u'2 overripe bananas (peeled and mashed)',
  u'2 tablespoons butter',
  u'2 tablespoons brown sugar',
  u'2 dashes of cinnamon',
  u'2 ripe bananas (peeled and sliced)',
  u'2 ounces dark rum\n'],
 u'publisher': u'Closet Cooking',
 u'publisher_url': u'http://closetcooking.com',
 u'recipe_id': u'35130',
 u'social_rank': 38.259222092243164,
 u'source_url': u'http://www.closetcooking.com/2010/05/bananas-foster-pancakes.html',
 u'title': u'Bananas Foster Pancakes'}

### Scrape ingredients from a Food2Fork.com page ###

In [17]:
def getIngredients(json_obj):
    r = getRecipe(json_obj['recipe_id'])
    return [i.strip('\n') for i in r['ingredients']]

In [23]:
ingredients = getIngredients(recipes[0])
ingredients

[u'4 small chicken breasts, pounded thin',
 u'salt and pepper to taste',
 u'4 jalapenos, diced',
 u'4 ounces cream cheese, room temperature',
 u'1 cup cheddar cheese, shredded',
 u'8 slices bacon']

## Use NLTK to parse amounts out from ingredients ##

In [37]:
from nltk import word_tokenize

In [33]:
[ingredient for ingredient in ingredients]

[u'4 small chicken breasts, pounded thin',
 u'salt and pepper to taste',
 u'4 jalapenos, diced',
 u'4 ounces cream cheese, room temperature',
 u'1 cup cheddar cheese, shredded',
 u'8 slices bacon']

In [41]:
pos = [nltk.pos_tag(word_tokenize(ingredient)) for ingredient in ingredients]
pos[0]

[(u'4', 'CD'),
 (u'small', 'JJ'),
 (u'chicken', 'NN'),
 (u'breasts', 'NNS'),
 (u',', ','),
 (u'pounded', 'VBD'),
 (u'thin', 'JJ')]